In [1]:
import pandas as pd
data = pd.read_csv('news_articles.csv')
data = data.iloc[:20000]
data.shape

(20000, 2)

In [2]:
data['tokens_text'] = data['text'].str.split(' ')
data
# print(data['tokens_text'].head(1).tolist()[0][:3])

,docno,text,tokens_text
0,FBIS3-40266,languag romanian articl typecso text draft law...,"[languag, romanian, articl, typecso, text, dra..."
1,FBIS3-40267,languag romanian articl typecso articl stelian...,"[languag, romanian, articl, typecso, articl, s..."
2,FBIS3-40268,languag slovak articl typebfn commentari juliu...,"[languag, slovak, articl, typebfn, commentari,..."
3,FBIS3-40269,languag slovak articl typecso interview pavol ...,"[languag, slovak, articl, typecso, interview, ..."
4,FBIS3-40270,languag slovak articl typebfn ta sr report ste...,"[languag, slovak, articl, typebfn, ta, sr, rep..."
...,...,...,...
19995,FBIS3-9283,languag serbocroatian articl typebfn text the ...,"[languag, serbocroatian, articl, typebfn, text..."
19996,FBIS3-9284,languag serbocroatian articl typebfn interview...,"[languag, serbocroatian, articl, typebfn, inte..."
19997,FBIS3-9285,languag serbocroatian articl typebfn report na...,"[languag, serbocroatian, articl, typebfn, repo..."
19998,FBIS3-9286,languag serbocroatian articl typebfn passag wi...,"[languag, serbocroatian, articl, typebfn, pass..."


In [3]:
data['tokens_text'].head(1).tolist()[0][:3]

['languag', 'romanian', 'articl']

In [4]:
from gensim.models import Phrases

In [5]:
tokens = data['tokens_text'].tolist()
bigram_model = Phrases(tokens)
trigram_model = Phrases(bigram_model[tokens], min_count=1)
tokens = list(trigram_model[bigram_model[tokens]])

In [6]:
from gensim import corpora

In [7]:
dictionary_LDA = corpora.Dictionary(tokens)
dictionary_LDA.filter_extremes(no_below=3)
corpus = [dictionary_LDA.doc2bow(tok) for tok in tokens]

In [8]:
from gensim import models
import numpy as np

In [9]:
np.random.seed(123456)
num_topics = 10
%time lda_model = models.LdaModel(corpus, num_topics=num_topics, \
                                  id2word=dictionary_LDA, \
                                  passes=4, alpha=[0.01]*num_topics, \
                                  eta=[0.01]*len(dictionary_LDA.keys()))

Wall time: 58.6 s


In [10]:
for i,topic in lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=20):
    print(str(i)+": "+ topic)
    print()

0: 0.033*"parti" + 0.019*"govern" + 0.015*"elect" + 0.012*"said" + 0.011*"polit" + 0.010*"vote" + 0.009*"would" + 0.009*"member" + 0.009*"new" + 0.008*"support" + 0.007*"candid" + 0.007*"deputi" + 0.007*"leader" + 0.006*"coalit" + 0.006*"presid" + 0.006*"prime_minist" + 0.006*"group" + 0.005*"also" + 0.005*"propos" + 0.005*"parliament"

1: 0.031*"us" + 0.027*"unit_state" + 0.025*"japan" + 0.016*"said" + 0.014*"north_korea" + 0.009*"govern" + 0.008*"japanes" + 0.007*"dprk" + 0.006*"offici" + 0.006*"would" + 0.006*"south_korea" + 0.006*"issu" + 0.006*"report" + 0.006*"korea" + 0.006*"washington" + 0.005*"china" + 0.005*"pyongyang" + 0.004*"say" + 0.004*"may" + 0.004*"talk"

2: 0.016*"govern" + 0.016*"countri" + 0.011*"econom" + 0.010*"nation" + 0.010*"develop" + 0.009*"polici" + 0.007*"also" + 0.007*"state" + 0.007*"must" + 0.006*"economi" + 0.006*"import" + 0.006*"new" + 0.005*"thi" + 0.004*"system" + 0.004*"region" + 0.004*"intern" + 0.004*"need" + 0.004*"problem" + 0.004*"trade" + 0.0

In [20]:
lda_model[corpus[1999]]

[(0, 0.10520574),
 (1, 0.39561754),
 (2, 0.25438666),
 (4, 0.116994545),
 (6, 0.12746978)]

In [12]:
topics = [lda_model[corpus[i]] for i in range(len(data))]
topics

[[(0, 0.037210118),
  (2, 0.13651347),
  (4, 0.40816218),
  (5, 0.14941202),
  (6, 0.04033334),
  (7, 0.22822434)],
 [(0, 0.13745134),
  (2, 0.12015146),
  (3, 0.10374403),
  (4, 0.028974121),
  (6, 0.0320864),
  (7, 0.1663543),
  (8, 0.39929128),
  (9, 0.011857973)],
 [(0, 0.035196904),
  (1, 0.01872939),
  (3, 0.12753525),
  (6, 0.2669418),
  (7, 0.044996236),
  (8, 0.5064737)],
 [(0, 0.14909203),
  (2, 0.20060909),
  (4, 0.02707456),
  (5, 0.038952358),
  (6, 0.027220206),
  (8, 0.55700284)],
 [(4, 0.6576432), (6, 0.31060478), (7, 0.03121081)],
 [(1, 0.01998789), (4, 0.52348655), (7, 0.38558832), (8, 0.07052179)],
 [(2, 0.33571744), (3, 0.22437415), (4, 0.14797033), (6, 0.29077822)],
 [(2, 0.17414819),
  (4, 0.10493288),
  (6, 0.39272055),
  (7, 0.12633063),
  (8, 0.16588438),
  (9, 0.035355702)],
 [(0, 0.16349603),
  (3, 0.03491271),
  (4, 0.21385762),
  (5, 0.021997822),
  (6, 0.56525475)],
 [(0, 0.3154083),
  (1, 0.012491803),
  (2, 0.4303189),
  (3, 0.033695098),
  (4, 0.0645029

In [13]:
def topics_document_to_dataframe(topics_document, num_topics):
    res = pd.DataFrame(columns=range(num_topics))
    for topic_weight in topics_document:
        res.loc[0, topic_weight[0]] = topic_weight[1]
    return res
print (num_topics)

10


In [14]:
for topic_document in topics:
    print (topics_document_to_dataframe(topic_document, num_topics))

         0    1         2    3         4         5         6         7    8  \
0  0.03721  NaN  0.136513  NaN  0.408162  0.149412  0.040333  0.228224  NaN   

     9  
0  NaN  
          0    1         2         3         4    5         6         7  \
0  0.137451  NaN  0.120151  0.103744  0.028974  NaN  0.032086  0.166354   

          8         9  
0  0.399291  0.011858  
          0         1    2         3    4    5         6         7         8  \
0  0.035197  0.018729  NaN  0.127535  NaN  NaN  0.266942  0.044996  0.506474   

     9  
0  NaN  
          0    1         2    3         4         5        6    7         8  \
0  0.149092  NaN  0.200609  NaN  0.027075  0.038952  0.02722  NaN  0.557003   

     9  
0  NaN  
     0    1    2    3         4    5         6         7    8    9
0  NaN  NaN  NaN  NaN  0.657643  NaN  0.310605  0.031211  NaN  NaN
     0         1    2    3         4    5    6         7         8    9
0  NaN  0.019988  NaN  NaN  0.523487  NaN  NaN  0.385588  0.07

In [17]:
# Final LDA feature extraction matrix with documents as rows and 10 topics as column (can change count of topics by num_topics variable value)
document_topic = pd.concat([topics_document_to_dataframe(topics_document, num_topics=num_topics) for topics_document in topics]) \
  .reset_index(drop=True).fillna(0)

In [22]:
document_topic.head()

,0,1,2,3,4,5,6,7,8,9
0,0.037210,0.000000,0.136513,0.000000,0.408162,0.149412,0.040333,0.228224,0.000000,0.000000
1,0.137451,0.000000,0.120151,0.103744,0.028974,0.000000,0.032086,0.166354,0.399291,0.011858
2,0.035197,0.018729,0.000000,0.127535,0.000000,0.000000,0.266942,0.044996,0.506474,0.000000
3,0.149092,0.000000,0.200609,0.000000,0.027075,0.038952,0.027220,0.000000,0.557003,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.657643,0.000000,0.310605,0.031211,0.000000,0.000000


In [19]:
document_topic.iloc[1999][:]

0    0.105199
1    0.395616
2    0.254384
3    0.000000
4    0.116988
5    0.000000
6    0.127486
7    0.000000
8    0.000000
9    0.000000
Name: 1999, dtype: float64

In [21]:
# Which document are about topic 4
document_topic.sort_values(4, ascending=False)[4].head(20)

14339    0.999489
15875    0.999461
18827    0.999343
2439     0.999326
15948    0.999262
15919    0.999197
15947    0.999126
9925     0.999051
535      0.999028
16248    0.998942
16227    0.998929
10341    0.998915
17710    0.998889
15745    0.998801
15802    0.998784
16293    0.998751
15793    0.998734
15765    0.998715
8661     0.998715
15776    0.998677
Name: 4, dtype: float64